In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
import folium
# !pip install pandas # 파이썬 데이터 처리 라이브러리
from pandas.io.json import json_normalize # 파이썬 딕셔너리 타입의 데이터를 판다스 데이터프레임 타입으로 변환한다.

# 전국 또는 특정 지역의 스타벅스 매장 위치 정보를 가져와서 지도위에 표시하기
requests로 스타벅스 매장의 위치 정보를 가져와서 딕셔너리 타입으로 변환한다.  
json_normalize() 함수로 json 타입의 데이터가 변환된 파이썬 딕셔너리를 판다스 데이터프레임으로 변환한다.  
folium 모듈을 사용해서 지도를 표시하고 지도위의 스타벅스 매장 위치에 마커를 표시한다.

In [2]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
req = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '01',
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 600,
    'set_date': ''
})
starbucks = req.json()
print(type(starbucks))

<class 'dict'>


In [3]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
starbucks_df = json_normalize(starbucks, 'list') # 2번째 인수로 데이터프레임으로 변환할 딕셔너리의 key를 적어준다.
print(type(starbucks_df)) # <class 'pandas.core.frame.DataFrame'>
print(len(starbucks_df))
print(starbucks_df.columns)

<class 'pandas.core.frame.DataFrame'>
585
Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       't22', 't21', 'p90', 't05', 't30', 't36', 't27', 't29', 't43', 't48'],
      dtype='object', length=129)


In [5]:
starbucks_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t22,t21,p90,t05,t30,t36,t27,t29,t43,t48
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
581,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
582,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
583,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터프레임을 만든다.
# s_name       => 지점이름
# sido_code    => 시도코드
# sido_name    => 시도이름
# gugun_code   => 구군코드
# gugun_name   => 구군이름
# doro_address => 도로명주소
# lat          => 위도
# lot          => 경도

In [8]:
# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적어도 되지만 얻어올 컬럼이
# 2개 이상일 경우 얻어올 컬럼 이름을 리스트로 만들어서 얻어와야 한다.
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.514132,127.020563
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
580,중랑구청,01,서울,0125,중랑구,서울특별시 중랑구 신내로 72,37.60538908,127.0957558
581,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
582,상봉역,01,서울,0125,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
583,묵동,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633


In [9]:
# dtypes 속성으로 판다스 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
# 판다스는 문자열 데이터 타입을 object라 부른다.
starbucks_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [11]:
# info() 메소드를 사용해도 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다.
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585 entries, 0 to 584
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        585 non-null    object
 1   sido_code     585 non-null    object
 2   sido_name     585 non-null    object
 3   gugun_code    585 non-null    object
 4   gugun_name    585 non-null    object
 5   doro_address  585 non-null    object
 6   lat           585 non-null    object
 7   lot           585 non-null    object
dtypes: object(8)
memory usage: 36.7+ KB


In [12]:
# astype() 메소드로 위도, 경도의 데이터 타입을 object에서 float로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.dtypes

s_name           object
sido_code        object
sido_name        object
gugun_code       object
gugun_name       object
doro_address     object
lat             float64
lot             float64
dtype: object

In [13]:
# 종각점
starbucks_df_map[starbucks_df_map['s_name'] == '종각']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
435,종각,01,서울,0118,종로구,서울특별시 종로구 종로 64 (종로2가),37.569918,126.984528


In [14]:
# 종로관철점
starbucks_df_map[starbucks_df_map['s_name'] == '종로관철']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
437,종로관철,01,서울,0118,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)",37.569058,126.986013


In [17]:
# 영풍문고점
starbucks_df_map[starbucks_df_map['s_name'] == '종로구청']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
422,종로구청,01,서울,0118,종로구,서울특별시 종로구 삼봉로 71 (수송동),37.572662,126.981138


In [27]:
starbucksMap = folium.Map(location=[37.5700249, 126.9836398], zoom_start=17)
popup = folium.Popup('종각점', max_width=300)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로관철점', max_width=300)
folium.Marker(location=[37.569058, 126.986013], popup=popup).add_to(starbucksMap)
popup = folium.Popup('종로구청점', max_width=300)
folium.Marker(location=[37.572662, 126.981138], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap1.html')
starbucksMap

In [30]:
# 서울특별시에 위치한 스타벅스 전 지점의 위치를 지도위에 표시한다.
# 지도를 작성할 때 지도의 중심으로 설정할 위치가 애매할 경우 지도위에 표시할 모든 스타벅스 전 지점의 위도와 경도의 평균을
# 중심으로 하는 지도를 만들어 사용하면 된다.
# 판다스 데이터프레임에 저장된 데이터의 평균은 mean() 메소드를 사용해서 계산한다.
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=12)
# iterrows() 메소드는 판다스 데이터프레임에 저장된 인덱스와 데이터를 리턴한다.
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap2.html')
starbucksMap

In [36]:
# 서울특별시 종로구 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
req = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '',
    'p_gugun_cd': '0118',
    'in_biz_cd': '',
    'iend': 600,
    'set_date': ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=14)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
starbucksMap

In [40]:
# 경기도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
req = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '08',
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 600,
    'set_date': ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=10)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap4.html')
starbucksMap

In [44]:
# 제주도 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
req = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '16',
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 600,
    'set_date': ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=11)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap4.html')
starbucksMap

In [45]:
# 전국 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
req = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '',
    'p_gugun_cd': '',
    'in_biz_cd': '',
    'iend': 1800,
    'set_date': ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=8)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap4.html')
starbucksMap

In [49]:
# 경기도 양주 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=8K3XF06R9Q'
req = requests.post(targetSite, data={
    'ins_lat': 37.563398,
    'ins_lng': 126.9863309,
    'p_sido_cd': '',
    'p_gugun_cd': '0821',
    'in_biz_cd': '',
    'iend': 600,
    'set_date': ''
})
starbucks = req.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[['s_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address',
                                'lat', 'lot']]
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)

starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=12)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup(data['s_name'] + '점, 주소: ' + data['doro_address'], max_width=500)
    folium.Marker(location=[data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap4.html')
starbucksMap